<!-- <img src="platia.png" width="150" height="150"> -->

<img src="https://www.cpqd.com.br/wp-content/uploads/2020/07/ico-plat1-268x300.png" width='150' height='150'>

# You Only Look Once (YOLO)

Este componente utiliza o modelo YOLO para classificação usando [Yolov4](https://pypi.org/project/yolov4/). <br>
Está é uma biblioteca que encapsula o modelo yolov4 com algumas variações, podendo utilizar o modelo completo, como também,a versão reduzida.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

In [ ]:
!pip install gdown

In [ ]:
# Download classes
!wget https://raw.githubusercontent.com/platiagro/projects/master/samples/default-yolo/coco.names

# Download weights
!gdown --id '15P4cYyZ2Sd876HKAEWSmeRdFl_j-0upi' --output 'yolov4.weights'

In [ ]:
%%writefile Model.py
from typing import List, Iterable, Dict, Union

import base64
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from yolov4.tf import YOLOv4


class Model:

    
    def __init__(self, dataset: str = None):
        
        model = load_model()
        self.names = model['names']
    
    
    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:

        yolo = YOLOv4()
        yolo.classes = "coco.names"
        yolo.make_model()
        yolo.load_weights("yolov4.weights", weights_type="yolo")
        
        im_bytes = base64.b64decode(X[0,0])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        frame = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        bboxes = yolo.predict(frame)
        
        return np.array(bboxes)

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "corn_image",
            "ftype": "categorical",
            "values": ["..."]        
        }
        
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json", total=8)